<a href="https://colab.research.google.com/github/Ak-Gautam/efficient_llm_fine_tunes/blob/main/llama3.1/FT_llama3_1_8b_openhermes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes
!pip install -U wandb
!pip install tqdm

In [ ]:
from transformers import Trainer
from accelerate import DistributedDataParallelKwargs

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [ ]:
from huggingface_hub import login
login(token = '')

wandb.login(key='')
run = wandb.init(
    project='Fine-tune Llama 3.1 8B on Chat data',
    job_type="training",
    anonymous="allow"
)

In [ ]:
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
dataset_name = "glouriousgautam/openhermes116k_balanced"
new_model = "llama-3-8b-hermes-chat-In"

In [ ]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

In [ ]:
from datasets import load_dataset
dataset = load_dataset(dataset_name, split = "train")
dataset = dataset.shuffle(seed=42)

In [ ]:
# Configure the chat template

tokenizer.token_mapping = {"role": "from", "content": "value", "user": "human", "assistant": "gpt"}

def format_chat_template(example):
    # Prepare the conversation in the format expected by apply_chat_template
    conversation = [
        {"role": msg["from"], "content": msg["value"]}
        for msg in example["conversations"]
    ]

    # Apply the chat template
    example["text"] = tokenizer.apply_chat_template(conversation, tokenize=False)

    return example

dataset = dataset.map(format_chat_template)
#print(dataset[5]["text"])

In [ ]:
dataset = dataset.train_test_split(test_size=0.03)

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    #num_train_epochs=1,
    max_steps=550,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=720,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()
model.config.use_cache = True

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What do you think about black holes?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False,
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True,
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150,
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
new_model = "glouriousgautam/llama-3.1-8b-hermes-In"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()

In [ ]:
messages = [{"role": "user", "content": "Write a short story about a strawberry that can ride a unicorn."}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=220, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

In [ ]:
model.save_pretrained("llama-3.1-8b-hermes-Instruct")
tokenizer.save_pretrained("llama-3.1-8b-hermes-Instruct")

In [ ]:
model.push_to_hub("llama-3.1-8b-hermes-Instruct", use_temp_dir=False)
tokenizer.push_to_hub("llama-3.1-8b-hermes-Instruct", use_temp_dir=False)